# **Scraping the Exhibitor List**
PAX just dropped [the 2025 exhibitor list](https://east.paxsite.com/en-us/expo-hall.html); within this notebook, I'll scrape it! 

# Setup
The cells below will help to set up the rest of the notebook. 

I'll start by configuring the kernel that's running this notebook:

In [ ]:
# Change the cwd
%cd ..

# Enable the autoreload module
%load_ext autoreload
%autoreload 2

# Load the environment variables
from dotenv import load_dotenv
load_dotenv(override=True)

Next, I'm going to import the necessary modules:

In [45]:
# General imports
import random
import re
from time import sleep

# Third-party imports
from tqdm import tqdm
import pandas as pd
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright

# Scraping the Exhibitor List
The Exhibitor List is actually loaded dynamically via JavaScript after the initial page load. Because of this, I've just used the Chrome inspector to copy the HTML data *after* the load; I can scrape from this list.

In [37]:
# Load the HTML from the saved file
with open("data/full-exhibitor-list-html.txt", "r", encoding="utf-8") as file:
    html_content = file.read()


# Define a method to scrape the exhibitors
def scrape_exhibitors(html_content):
    """
    Extract exhibitor information from the PAX East exhibitor list HTML.

    Args:
        html_content (str): The HTML content of the exhibitor list page

    Returns:
        exhibitor_list (List[dict]): A list of dictionaries containing exhibitor information
    """
    soup = BeautifulSoup(html_content, "html.parser")
    exhibitors = []

    # Find all exhibitor entries
    exhibitor_entries = soup.find_all("div", class_="exhibitor-entry")

    for entry in exhibitor_entries:
        exhibitor = {}

        # Get image URL
        img_tag = entry.find("img")
        exhibitor["image_url"] = img_tag["src"] if img_tag else None

        # Get name
        name_tag = entry.find("div", class_="exhibitor-name").find("a")
        exhibitor["name"] = name_tag.text.strip() if name_tag else None

        # Get description
        desc_tag = entry.find("div", class_="exhibitor-description")
        exhibitor["description_excerpt"] = desc_tag.text.strip() if desc_tag else None

        # Get booths
        location_tag = entry.find("div", class_="exhibitor-location")
        booth_text = location_tag.text if location_tag else ""
        # Extract booth numbers using regex
        booth_numbers = re.findall(r"\d+", booth_text)
        exhibitor["booths"] = booth_numbers

        # Get details page link
        link_tag = entry.find("a", class_="gtExhibitorLink")
        if link_tag:
            # Fix the URL by removing the duplicate domain
            href = link_tag["href"]
            if href.startswith("https://east.paxsite.com"):
                exhibitor["details_url"] = href
            else:
                exhibitor["details_url"] = f"https://east.paxsite.com{href}"
        else:
            exhibitor["details_url"] = None

        exhibitors.append(exhibitor)

    return exhibitors


# Call the function to scrape exhibitors
exhibitors = scrape_exhibitors(html_content)

# Convert the list of exhibitors to a DataFrame
exhibitor_df = pd.DataFrame(exhibitors)

# Add a column for the number of booths
exhibitor_df["n_booths"] = exhibitor_df["booths"].apply(
    lambda x: len(x) if isinstance(x, list) else 0
)

What does this scraped data look like?

In [ ]:
exhibitor_df.head(5)

# Scraping Exhibitor Details
Next up: for each of the exhibitors on the list, I'm going to scrape some details related to them.

The method below will use a headless Playwright client to scrape the details:

In [58]:
async def scrape_exhibitor_details(url):
    from tenacity import (
        retry,
        stop_after_attempt,
        wait_exponential,
        retry_if_exception_type,
    )

    @retry(
        stop=stop_after_attempt(3),
        wait=wait_exponential(multiplier=1, min=2, max=10),
        retry=retry_if_exception_type((TimeoutError, Exception)),
        reraise=True,
    )
    async def _scrape_with_retry(page, url):
        # Navigate to the URL with a longer timeout
        await page.goto(url, timeout=30000, wait_until="networkidle")

        # Wait for the main content to load with a longer timeout
        await page.wait_for_selector(".innovation__base", timeout=20000)

        # Add a small delay to ensure JavaScript has fully executed
        await page.wait_for_timeout(random.uniform(1000, 2000))

        # Create a result dictionary with default values
        result = {
            "details_url": url,
            "booth": None,
            "name": None,
            "description": None,
            "website_url": None,
            "shop_url": None,
            "playable_games": [],
        }

        # Extracting the booth number
        booth_element = await page.query_selector("h3.showroomLocation")
        if booth_element:
            booth_text = await booth_element.inner_text()
            # Extract just the number part after "Booth:"
            if ":" in booth_text:
                result["booth"] = booth_text.split(":", 1)[1].strip()

        # Extracting the name from the logo alt text
        logo_element = await page.query_selector("#showroomLogo img")
        if logo_element:
            result["name"] = await logo_element.get_attribute("alt")
        # Fallback to data attribute if available
        if not result["name"]:
            logo_link = await page.query_selector(".showroom-logo-modal")
            if logo_link:
                result["name"] = await logo_link.get_attribute("data-exhib-full-name")

        # Extracting the description
        desc_element = await page.query_selector("div#showroomDesc p")
        if desc_element:
            result["description"] = await desc_element.inner_text()

        # Extracting the website URL
        website_element = await page.query_selector("li.website a")
        if website_element:
            result["website_url"] = await website_element.get_attribute("href")

        # Extracting the shop URL
        shop_element = await page.query_selector("li.store a")
        if shop_element:
            result["shop_url"] = await shop_element.get_attribute("href")

        # Extracting playable games
        game_elements = await page.query_selector_all(
            "ul.singleSpecialList li.singleSpecial"
        )
        for game_element in game_elements:
            game_data = {}

            title_element = await game_element.query_selector(
                "div.gtSlideSpecial-title"
            )
            if title_element:
                game_data["name"] = await title_element.inner_text()
            else:
                game_data["name"] = None

            image_element = await game_element.query_selector(
                "div.gtSlideSpecial-image"
            )
            if image_element:
                image_style = await image_element.get_attribute("style")
                if image_style and "url(" in image_style:
                    url_part = image_style.split("url(")[1].split(")")[0]
                    game_data["image_url"] = url_part.strip("'\"")
                else:
                    game_data["image_url"] = None
            else:
                game_data["image_url"] = None

            link_element = await game_element.query_selector("a")
            if link_element:
                game_data["merch_link"] = await link_element.get_attribute(
                    "data-backupurl"
                )
            else:
                game_data["merch_link"] = None

            result["playable_games"].append(game_data)

        return result

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        try:
            result = await _scrape_with_retry(page, url)
        except Exception:
            # If all retries fail, return empty result with URL
            result = {
                "details_url": url,
                "booth": None,
                "name": None,
                "description": None,
                "website_url": None,
                "shop_url": None,
                "playable_games": [],
            }

        await browser.close()
        return result

For each of the exhibitors I've found, I'll scrape their details:

In [ ]:
exhibitor_details_list = []
for row in tqdm(
    iterable=list(exhibitor_df.itertuples()),
    desc="Scraping exhibitor details",
):
    # Skip if the URL is None
    if row.details_url is None:
        continue

    # Otherwise, scrape the details
    exhibitor_details_list.append(await scrape_exhibitor_details(row.details_url))

    # Sleep for a random amount of time
    sleep_time = random.uniform(3, 8)
    sleep(sleep_time)

# Make a DataFrame from the list of dictionaries
exhibitor_details_df = pd.DataFrame(exhibitor_details_list)

What does this look like? 

In [ ]:
exhibitor_details_df.sample(5)